In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# from matplotlib.colors import LogNorm
from matplotlib.gridspec import GridSpec
from scipy.integrate import solve_ivp

# Corrected imports
from defs import active_volume, ring_calculation, add_lengths_to_df, Ring, sim_params # Added sim_params
from magpylib.magnet import Cylinder, CylinderSegment # Imported from magpylib

mpl.rcParams["lines.linewidth"] = 2.5
mpl.rcParams["font.size"] = 20
# mpl.rcParams('legend',fontsize=20) # using a size in points
plt.rc("legend", fontsize=17)  # using a size in points

In [ ]:
# --- Parameter Definitions ---
ngeom_default = 41
ngeom_secondary = 18 # for R variation scan

# Default geometric parameters for the first set of scans (varying L)
R_default_scan1 = 20  # Default radius in mm
dR_default_list_scan1 = [2]  # Default dR list in mm
L_default_linspace_params_scan1 = (10, 90) # (start, end) for L linspace

# Default geometric parameters for the second set of scans (varying R)
L_default_scan2 = 50 # Default length in mm
dR_default_list_scan2 = [2] # Default dR list in mm
R_default_linspace_params_scan2 = (6, 40) # (start, end) for R linspace

# Filenames for saving data and plots
scan_l_vs_r_filename = "scan_l_vs_r_data.csv" # Renamed for clarity
parallelism_plot_filename = "parallelism_L_scan.png"
zsep_va_plot_filename = "zsep_va_L_scan.png"
lengths_plot_filename = "lengths_L_scan.png"
normed_lengths_plot_filename = "normed_lengths_L_scan.png"
rl_field_plot_filename = "RL_field_examples.png"
scan_r_vs_l_filename = "scan_r_vs_l_data.csv"
lengths_R_varied_plot_filename = "lengths_R_scan.png"

In [ ]:
ngeom = ngeom_default

In [ ]:
### magnetic geometry

# to plot or not to plot (scans)
plt_on = False

# prepare a dataframe header
df = pd.DataFrame(
    columns=["R", "L", "dR", "parallelism", "zsep_L", "va", "r0", "mr", "length"]
)

# Use parameterized values for the first scan (varying L)
R_scan1 = R_default_scan1

for dR_scan1 in dR_default_list_scan1:
    for L_scan1 in np.linspace(L_default_linspace_params_scan1[0], L_default_linspace_params_scan1[1], ngeom):  # length in mm
        # Comments for previous loop variations removed for clarity
        calc_output = ring_calculation(R_scan1, L_scan1, dR_scan1, plt_on)  # calculate the field
        rlines = calc_output.r0_lines
        Brinterp = calc_output.interp_r_func
        Bzinterp = calc_output.interp_z_func
        parallel = calc_output.pa_val
        zsep = calc_output.zsep
        # zz, rr, and ring (magnet object) from calc_output are not directly used here for integration input
        df = integration(
            R_scan1, L_scan1, dR_scan1, df, rlines, Brinterp, Bzinterp, parallel, zsep, plt_on
        )  # generate parameter df

In [ ]:
# Fig. 4


def format_func(value, tick_number):
    """pi as ticks"""
    # find number of multiples of pi/2
    N = int(np.round(2 * value / np.pi))
    if N == 0:
        return "0"
    elif N == 1:
        return r"$\pi/2$"
    elif N == 2:
        return r"$\pi$"
    elif N % 2 > 0:
        return rf"${N}\pi/2$"
    else:
        return rf"${N // 2}\pi$"


# analytical result
pb = df.R[0] * np.sqrt(6)  # parallelB value for R = 20mm


fig, ax = plt.subplots(figsize=(10, 7))

pp = [0.1, 0.3, 0.5, 0.9]

pas = np.array(df.parallelism.tolist(), dtype=float)
num = pas.shape[1]
# for p in range(num - 1):
#    plt.plot(df.L, np.sin(pas[:, p]), label="p = " + str(pp[p]))  # *180/np.pi)

# plt.plot(df.L, np.tan(pas[:, 0]))
plt.plot(df.L, np.sin(pas[:, 0]))
plt.axvline(x=pb, color="tab:olive")

# plt.yticks([-np.pi,0,np.pi])
plt.xlabel("L / mm")
# plt.ylabel(r"$B_r$ / $B_z$ ")
plt.ylabel(r"$\sin (\alpha)$ ")
# plt.legend()
ax.ticklabel_format(axis="y", scilimits=[-3, 3])

# ax.yaxis.set_major_locator(plt.MultipleLocator(np.pi))
# ax.yaxis.set_minor_locator(plt.MultipleLocator(np.pi / 2))
# ax.yaxis.set_major_formatter(plt.FuncFormatter(format_func))

fig.tight_layout()
plt.grid()
plt.savefig(parallelism_plot_filename, dpi=300)

In [ ]:
# Fig. 5

fig, ax = plt.subplots(figsize=(10, 7), dpi=300)
ax2 = ax.twinx()

lns1 = ax.plot(df.L, df.zsep_L / df.L, label=r"$\widetilde{z}$")
ax.set_xlabel("L / mm")
# ax.set_ylabel("$(z_{sep}-L/2)/L$ / mm")
ax.set_ylabel(r"$\widetilde{z}$ / mm")
ax.grid()

# select only the uppermost field line from the df for each geometry
# The DataFrame `df` stores results for `ngeom` different geometries (L values).
# For each geometry, `sim_params.nlines + 1` field lines are integrated.
# `nn` represents the number of field lines per geometry.
nn = sim_params.nlines + 1 # Address TODO: Use parameter from sim_params
line = []
# This loop creates a list of indices to select the first field line (index 0 within its group)
# for each of the `ngeom` geometries.
for rrr in range(ngeom):  # Iterate through each geometry scanned
    line.append(0 + (nn) * rrr) # Index of the first field line for the rrr-th geometry

# plt.plot(df.L[line], 1 - (df.va[line] / (R_default_scan1*R_default_scan1*df.L[line]/2))) # Example with R_default_scan1
lns2 = ax2.plot(df.L[line], df.va[line], "r", label=r"$\widetilde{V}$")  # /(R*R*0.5*L))
# ax2.set_ylabel("$\widetilde{V}$ = $V_{active}$ / $V_{total}$")
ax2.set_ylabel(r"$\widetilde{V}$")

lns = lns1 + lns2
labs = [l.get_label() for l in lns]
ax.legend(lns, labs)
plt.tight_layout()

plt.savefig(zsep_va_plot_filename, dpi=300, bbox_inches="tight")

In [ ]:
df.to_csv(scan_l_vs_r_filename)

df = add_lengths_to_df(df)  # add lengths to df

# integral values for each thruster geometry
dfsum = df.groupby(["R", "L"]).sum().reset_index()
dfsum


In [ ]:
# Fig. 7


def plot_lengths(df, dfsum):
    """plot various length definitions"""
    pb = df.R[0] * np.sqrt(6)  # parallelB value for R = 20mm

    fig, ax = plt.subplots(3, 1, figsize=(8, 8), sharex=True)
    nlines_for_avg = sim_params.nlines + 1 # Number of lines used for averaging in dfsum
    rhos = [df.r0[2], df.r0[10], df.r0[17]] # Example r0 values for specific line plots
    label = "average"

    # plot all lengths for all parameters, averaged over field lines
    ax[0].plot(dfsum.L, dfsum.length / nlines_for_avg, label=label)
    ax[0].axvline(x=pb, color="tab:olive")
    ax[0].set_ylabel(r"$\ell$ / a.u.")
    ax[1].plot(dfsum.L, dfsum.mr / nlines_for_avg, label=label)
    ax[1].axvline(x=pb, color="tab:olive")
    ax[1].set_ylabel(r"$R_m$")

    # exponentially weighted lengths, averaged over field lines
    ax[2].plot(dfsum.L, dfsum["r0_l_mr_exp_r0"] / nlines_for_avg, label=label)
    ax[2].axvline(x=pb, color="tab:olive")
    ax[2].set_ylabel(r"$\tau$ / a.u.")
    # $\hat{\ell} = \ell \cdot \sqrt{R_m} \cdot \rho_0 \cdot e^{-\rho_0}$

    # for selected lines
    ls = ["-", ":", "--"]
    for i, l in enumerate(rhos):
        dff = df.where(np.abs(df.r0 - l) < 0.2).dropna()
        # plot all lengths for all parameters

        ax[0].plot(
            dff.L,
            dff.length,
            label=str(np.round(l, decimals=1)) + " mm",
            linestyle=ls[i],
        )
        ax[1].plot(
            dff.L, dff.mr, label=str(np.round(l, decimals=1)) + " mm", linestyle=ls[i]
        )
        ax[2].plot(
            dff.L,
            dff["r0_l_mr_exp_r0"],
            label=str(np.round(l, decimals=1)) + " mm",
            linestyle=ls[i],
        )

    # ax[0,0].legend()
    # ax[0,1].legend()
    # ax[1,0].legend()
    ax[1].legend()
    ax[2].set_xlabel("L / mm")


plot_lengths(df, dfsum)
plt.savefig(lengths_plot_filename, dpi=300)

In [ ]:
ndf = df.copy()  # normed df ->
ndf = add_lengths_to_df(ndf, ndf.L)

ndfsum = ndf.groupby(["R", "L"]).sum().reset_index()
ndfsum

plot_lengths(ndf, ndfsum)
plt.savefig(normed_lengths_plot_filename, dpi=300)

In [ ]:
### Fig. 3

# to plot or not to plot (scans)
plt_on = True

# prepare a dataframe header
df = pd.DataFrame(
    columns=["R", "L", "dR", "parallelism", "zsep_L", "va", "r0", "mr", "length"]
)


fig = plt.figure(constrained_layout=True, figsize=(11, 8), dpi=300)
fig.set_tight_layout(True)

gs = GridSpec(2, 6, figure=fig)
ax1 = fig.add_subplot(gs[0, :2])
ax2 = fig.add_subplot(gs[0, 2:])
ax3 = fig.add_subplot(gs[-1, :])

axes = [ax1, ax2, ax3]
label = ["(a)", "(b)", "(c)"]

# fixed R, example values for L and dR for plotting field lines
R_example = R_default_scan1 # Use the same R as the first scan for consistency
dR_example_list = dR_default_list_scan1 # Use the same dR as the first scan
L_example_list = [10, 50, 90] # Example L values for specific field plots

for dR_ex in dR_example_list:
    for i, L_ex in enumerate(L_example_list):
        m = Ring(R_example, L_ex, dR_ex, plt_on, axes[i])
        df = integration(
            m.R, m.L, m.dR, df, m.r0, m.bri, m.bzi, m.pa, m.zsep, plt_on, axes[i]
        )  # generate parameter df (note: df is being appended globally, might be unintended for this example section)
        axes[i].plot(m.zsep, 0, "k*", markersize=14)
        axes[i].set_title(label[i], fontfamily="serif", loc="left", fontsize="medium")


plt.savefig(rl_field_plot_filename, dpi=300, bbox_inches="tight")

In [ ]:
### magnetic geometry - Scan 2: Varying R
ngeom_R_scan = ngeom_secondary # Use parameterized number of geometry points for R scan

# to plot or not to plot (scans)
plt_on = False

# prepare a dataframe header
dfr = pd.DataFrame(
    columns=["R", "L", "dR", "parallelism", "zsep_L", "va", "r0", "mr", "length"]
)

# Use parameterized values for the second scan (varying R)
L_scan2 = L_default_scan2

for dR_scan2 in dR_default_list_scan2:
    for R_scan2 in np.linspace(R_default_linspace_params_scan2[0], R_default_linspace_params_scan2[1], ngeom_R_scan):
        # Comments for previous loop variations removed for clarity
        calc_output = ring_calculation(R_scan2, L_scan2, dR_scan2, plt_on)  # calculate the field
        rlines = calc_output.r0_lines
        Brinterp = calc_output.interp_r_func
        Bzinterp = calc_output.interp_z_func
        parallel = calc_output.pa_val
        zsep = calc_output.zsep
        # zz, rr, and ring (magnet object) from calc_output are not directly used here for integration input
        dfr = integration(
            R_scan2, L_scan2, dR_scan2, dfr, rlines, Brinterp, Bzinterp, parallel, zsep, plt_on
        )  # generate parameter df

In [ ]:
dfr

In [ ]:
dfr = add_lengths_to_df(dfr)

# dfr
dfrsum = dfr.groupby(["R", "L"]).sum().reset_index()
dfrsum.to_csv(scan_r_vs_l_filename) # Save the summarized data for this scan
dfrsum # Display the dataframe

In [ ]:
# Fig. 8


def plot_lengths_R(df, dfsum):
    """plot various length definitions"""
    pb = df.L[0] / np.sqrt(6)  # parallelB value for L = 50mm

    fig, ax = plt.subplots(3, 1, figsize=(8, 8), sharex=True)

    nlines_for_avg_R_scan = sim_params.nlines + 1 # Number of lines used for averaging in dfsum
    # rhos defines indices of specific field lines to plot individually.
    # These are 0-indexed within each group of 'nlines_for_avg_R_scan' lines.
    rhos_indices = [2, 10, 17]  # Example indices of r0 within each geometry's set of field lines
    rlabel = ["0.85", "0.5", "0.1"] # Labels corresponding to relative r0/R positions

    label = "average"

    # Plotting average values from dfsum
    ax[0].plot(dfsum.R, dfsum.length / nlines_for_avg_R_scan, label=label)
    ax[0].axvline(x=pb, color="tab:olive")
    ax[0].set_ylabel(r"$\ell$ / a.u.")
    ax[1].plot(dfsum.R, dfsum.mr / nlines_for_avg_R_scan, label=label)
    ax[1].axvline(x=pb, color="tab:olive")
    ax[1].set_ylabel("$R_m$")

    # exponentially weighted lengths, averaged
    ax[2].plot(dfsum.R, dfsum["r0_l_mr_exp_r0"] / nlines_for_avg_R_scan, label=label)
    ax[2].axvline(x=pb, color="tab:olive")
    ax[2].set_ylabel(r"$\tau$ / a.u.")

    # Plotting specific field lines from the original dfr DataFrame
    ls = ["-", ":", "--"]
    for i, line_index_in_group in enumerate(rhos_indices):
        # Construct a list of global indices for the specific field line across all R scans
        # Each R scan (geometry) has 'nlines_for_avg_R_scan' rows in dfr.
        # 'rrr' iterates through each R scan.
        # 'line_index_in_group' is the 0-based index of the desired field line within that R scan's group.
        line_global_indices = []
        for rrr in range(ngeom_R_scan):  # ngeom_R_scan is the number of R values scanned
            line_global_indices.append(line_index_in_group + (nlines_for_avg_R_scan * rrr))
        
        # Select rows from dfr using these global indices
        dff = df.iloc[line_global_indices]
        
        ax[0].plot(
            dff.R, dff.length, label=r"$r_0/R \approx $ " + str(rlabel[i]), linestyle=ls[i]
        )
        ax[1].plot(dff.R, dff.mr, label=r"$r_0/R \approx $ " + str(rlabel[i]), linestyle=ls[i])
        ax[2].plot(dff.R, dff["r0_l_mr_exp_r0"], label=r"$r_0/R \approx $ " + str(rlabel[i]), linestyle=ls[i])

    ax[0].legend()
    ax[2].set_xlabel("R / mm")


plot_lengths_R(dfr, dfrsum)
plt.savefig(lengths_R_varied_plot_filename, dpi=300)

In [ ]:
# CylinderSegment is shorter, use instead!

# ring from CylinderSegment
ring0 = CylinderSegment(magnetization=(0, 0, 1000), dimension=(2, 4, 1, 0, 360))

# ring with cut-out
inner = Cylinder(magnetization=(0, 0, -1000), dimension=(4, 1))
outer = Cylinder(magnetization=(0, 0, 1000), dimension=(8, 1))
ring1 = inner + outer

print("getB from Cylindersegment", ring0.getB((1, 2, 4)))
print("getB from Cylinder cut-out", ring1.getB((1, 2, 4)))